In [1]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
import requests  # url 요청
import time
from random import randint
import pandas as pd

In [2]:
dr = webdriver.Chrome()

In [3]:
dr.get("https://everytime.kr")
# 이후, 로그인 -> 이건 수작업으로 해야함!

In [4]:
dr.get("https://everytime.kr/lecture")

In [5]:
# 과목이름으로 강의평 검색
target_list = ['경영정보시스템및실습','경영과학(OR)및실습1','응용통계및연습1','응용통계및연습2','기술관리','생산관리',
               '유니버설디자인의이해','시스템경영공학종합설계','인간공학및실습','창의적융합디자인']
lecture_info_col_name = ['강의명','교수명','평균 평점','평점 참가자 수','평점 비율','과제량 비율','조모임 비율','학점 비율']
lecture_info = []
lecture_review_col_name = ['강의명', '교수명', '수강 시기', '강평']
lecture_review = []

for target in target_list:
    dr.get(f"https://everytime.kr/lecture/search?keyword={target}&condition=name")
    time.sleep(randint(1,5))

    # 과목이름 검색에서 해당 검색어가 포함되면 다른 과목이 나오기도 함 -> 전처리 필요 ex) 경영정보시스템 -> 경영정보시스템, 경영정보시스템및실습
    basic_lecture_info = [] # element: [강의명, 교수명]

    # 동일한 과목에 대한 교수님 많은 경우 고려... 스크롤 아래까지 내렸다가 다시 올려서 진행
    body = dr.find_element(By.CSS_SELECTOR, 'body')
    for k in range(2):
        for kk in range(30):
            body.send_keys(Keys.PAGE_DOWN)
        time.sleep(1)
        
    for k in range(2):
        for kk in range(30):
            body.send_keys(Keys.PAGE_UP)
        time.sleep(1)
    
    lecture_list = dr.find_elements(By.CLASS_NAME,"lecture")
    for k in range(1,len(lecture_list)+1):
        temp_lecture_name = dr.find_element(By.XPATH, f"/html/body/div/div/div[2]/a[{k}]/div[1]/span").text
        temp_lecture_prof = dr.find_element(By.XPATH, f"/html/body/div/div/div[2]/a[{k}]/div[2]").text
        basic_lecture_info.append([temp_lecture_name,temp_lecture_prof])

    # 과목에 대해서...
    for lecture_idx in range(len(lecture_list)):
        temp_lecture_info = [] # element : [강의명, 교수명, 평균 평점, 평점 참가자 수, [평점 비율], [과제량 비율], [조모임 비율], [학점 비율]]
        lecture = lecture_list[lecture_idx] # 특정 강의 Box 위치
        lecture_name = basic_lecture_info[lecture_idx][0] # 강의명
        lecture_prof = basic_lecture_info[lecture_idx][1] # 교수명
        temp_lecture_info.append(lecture_name)
        temp_lecture_info.append(lecture_prof)
        
        lecture.click()
        time.sleep(randint(1,5))
        # 여기서 교수명은 지난 몇년간 해당 강의를 안했을 경우, 다른 방식으로 접근해야함... ex) 이종석 교수님의 응통 -> 이전 과정에 미리 과목명, 교수명 확보

        # 강의평 없는 경우, 아래 코드 실행 불가능하므로 continue 조건, 이때 XPATH도 다름
        if dr.find_element(By.CLASS_NAME,"count").text == "(0개)":
            dr.back()
            time.sleep(randint(1,5))
            continue

        # 평균 평점 format_example = 3.24
        average_score = float(dr.find_element(By.XPATH, "/html/body/div/div/div[2]/div/section[2]/div[1]/div[1]/span[1]").text)
        temp_lecture_info.append(average_score)
        
        # 평점 참가자 수 format_example = 3
        part_num = int(dr.find_element(By.XPATH, "/html/body/div/div/div[2]/div/section[2]/div[1]/div[1]/span[3]").text[1:-2])
        temp_lecture_info.append(part_num)
        
        try:
            # 강의 평점별 비율 리스트 format_example = [30, 0, 0, 20, 50]   <= (5점 비율, ..., 1점 비율)
            score_ratio_list = []
            for k in range(1,6):
                temp_score_ratio = int(dr.find_element(By.XPATH,f"/html/body/div/div/div[2]/div/section[2]/div[1]/div[2]/div[{k}]/div[2]").text[:-1])
                score_ratio_list.append(temp_score_ratio)
            temp_lecture_info.append(score_ratio_list)

            # 과제량 비율 리스트 format_example = [30,20,50]   <= (없음 비율, 보통 비율, 많음 비율)
            hw_list = ["없음", "보통", "많음"]
            hw_ratio_list = []
            for k in range(1,4):
                temp_hw_ratio = int(dr.find_element(By.XPATH,f"/html/body/div/div/div[2]/div/section[2]/div[2]/div[1]/div/div[{k}]/div[2]").text[:-1])
                hw_ratio_list.append(temp_hw_ratio)
            temp_lecture_info.append(hw_ratio_list)    

            # 조모임 비율 리스트 format_example = [30,20,50]   <= (없음 비율, 보통 비율, 많음 비율)
            proj_ratio_list = []
            for k in range(1,4):
                temp_proj_ratio = int(dr.find_element(By.XPATH,f"/html/body/div/div/div[2]/div/section[2]/div[2]/div[2]/div/div[{k}]/div[2]").text[:-1])
                proj_ratio_list.append(temp_proj_ratio)
            temp_lecture_info.append(proj_ratio_list)

            # 성적 비율 리스트 format_exaple = [30,20,50]   <= (너그러움 비율, 보통 비율, 깐깐함 비율)
            grade_ratio_list = []
            grade_list = ["너그러움", "보통", "깐깐함"]
            for k in range(1,4):
                grade_ratio = int(dr.find_element(By.XPATH,f"/html/body/div/div/div[2]/div/section[2]/div[2]/div[3]/div/div[{k}]/div[2]").text[:-1])
                grade_ratio_list.append(grade_ratio)
            temp_lecture_info.append(grade_ratio_list)
        
        except:
            # 비율 관련 정보 X 경우
            try:
                score_ratio_list = None
                hw_ratio = dr.find_element(By.XPATH,"/html/body/div/div/div[2]/div/section[2]/div[2]/div[1]/div[1]/span").text
                proj_ratio = dr.find_element(By.XPATH,"/html/body/div/div/div[2]/div/section[2]/div[2]/div[1]/div[2]/span").text
                grade_ratio = dr.find_element(By.XPATH,"/html/body/div/div/div[2]/div/section[2]/div[2]/div[1]/div[3]/span").text
                temp_lecture_info.append(score_ratio_list)
                
                if hw_ratio == "없음":
                    hw_ratio_list = [100,0,0]
                elif hw_ratio == "보통":
                    hw_ratio_list = [0,100,0]
                else:
                    hw_ratio_list = [0,0,100]
                
                if proj_ratio == "없음":
                    proj_ratio_list = [100,0,0]
                elif proj_ratio == "보통":
                    proj_ratio_list = [0,100,0]
                else:
                    proj_ratio_list = [0,0,100]
                    
                if grade_ratio == "너그러움":
                    grade_ratio_list = [100,0,0]
                elif grade_ratio == "보통":
                    grade_ratio_list = [0,100,0]
                else:
                    grade_ratio_list = [0,0,100]
                
                temp_lecture_info.append(hw_ratio_list)
                temp_lecture_info.append(proj_ratio_list)
                temp_lecture_info.append(grade_ratio_list)
            
            # 비율 & 과제, 팀플, 성적 관련 정보 X 경우
            except:
                score_ratio_list = None
                hw_ratio_list = None
                proj_ratio_list = None
                grade_ratio_list = None
                temp_lecture_info.append(score_ratio_list)
                temp_lecture_info.append(hw_ratio_list)
                temp_lecture_info.append(proj_ratio_list)
                temp_lecture_info.append(grade_ratio_list)

        # lecture_info 업데이트
        lecture_info.append(temp_lecture_info)
        
        # 해당 강의 강의평 더보기 클릭
        dr.find_element(By.CLASS_NAME, "more").click()
        time.sleep(randint(1,5))

        # 스크롤 내리기 위해서 화면 한번 클릭
        dr.find_element(By.CLASS_NAME,'article_header').click()
        body = dr.find_element(By.CSS_SELECTOR, 'body')
        for k in range(1,part_num + 1):
            # 스크롤 내리기
            for n in range(3):
                body.send_keys(Keys.PAGE_DOWN)
            # 강평자 정보 추출 format_example = "22년 1학기 수강자", "22년 겨울 수강자"
            review_info = dr.find_element(By.XPATH, f"/html/body/div/div/div[2]/div/div[2]/div[{k}]/div[1]/div[1]/div[2]/span").text
            
            # 강평 후기 추출 format_example = "강의력 좋아요."
            review_data = dr.find_element(By.XPATH,f"/html/body/div/div/div[2]/div/div[2]/div[{k}]/div[2]").text
            lecture_review.append([lecture_name, lecture_prof, review_info, review_data])

        # 해당 강의 전체 정보 화면으로 돌아가기
        dr.back()
        time.sleep(randint(1,5))

        # 과목 검색 이후 화면으로 돌아가기
        dr.back()
        time.sleep(randint(1,5))
        

lecture_info_df = pd.DataFrame(lecture_info, columns=lecture_info_col_name)
lecture_review_df = pd.DataFrame(lecture_review, columns=lecture_review_col_name)


KeyboardInterrupt: 

In [6]:
lecture_info_df = pd.DataFrame(lecture_info, columns=lecture_info_col_name)
lecture_review_df = pd.DataFrame(lecture_review, columns=lecture_review_col_name)

In [7]:
lecture_info_df

,강의명,교수명,평균 평점,평점 참가자 수,평점 비율,과제량 비율,조모임 비율,학점 비율
0,경영정보시스템및실습,김종모,4.36,11,"[36, 64, 0, 0, 0]","[0, 73, 27]","[0, 36, 64]","[73, 27, 0]"
1,경영정보시스템및실습,손미애,3.25,102,"[16, 25, 36, 14, 9]","[3, 28, 69]","[16, 41, 43]","[11, 48, 41]"


In [8]:
lecture_review_df

,강의명,교수명,수강 시기,강평
0,경영정보시스템및실습,김종모,22년 1학기 수강자,"재밌고 유익한 강의\n교수님 강의력도 좋고, 좋은 팀원들 만나서 팀플 잘 한듯"
1,경영정보시스템및실습,김종모,22년 1학기 수강자,이제는 손미애 교수님이 하시겟지만...\n김종모교수님 갓.... 그저 빛
2,경영정보시스템및실습,김종모,22년 1학기 수강자,교수님 말이 정말정말 빠르세요 필기가 불가능할 정도로 빠르심 근데 아캠에 강의저장을...
3,경영정보시스템및실습,김종모,22년 1학기 수강자,시험은 몇문제 빼고 괜찮은데 팀을 잘 만나야함. 교수님 굉장히 좋으시고 수업시간때 ...
4,경영정보시스템및실습,김종모,22년 1학기 수강자,교수님 말씀이 조금 빠르시지만 학점 미화.\n조별과제랑 시험 둘다 꽤 빡세서 엉엉 ...
5,경영정보시스템및실습,김종모,22년 1학기 수강자,성적은 잘 주시는 편인데 성적간 차이가 정말 촘촘하다. 중간기말 그저 그렇게 보고 ...
6,경영정보시스템및실습,김종모,22년 1학기 수강자,저는 팀원들 다 잘 만나서 개인 과제보다 팀플 점수가 더 높습니다. 마지막에 기말도...
7,경영정보시스템및실습,김종모,22년 1학기 수강자,케이스 스터디 2번\n텀프로젝트 1번\n중간 기말\n비중은 엇비슷한데 교수님이 말이...
8,경영정보시스템및실습,김종모,22년 1학기 수강자,교수님과 조교님들이 엄청 착하시다. 그리고 정보시스템과 그것을 활용한 분야를 넓게 ...
9,경영정보시스템및실습,김종모,22년 1학기 수강자,교수님이랑 조교님 다 착하시고 학생의견 반영도 잘해주시는데 수업 내용이 너무 방대해요ㅠㅠ


In [38]:
lecture_info_df.to_csv('./Downloads/lecture_info.csv',encoding='utf-8-sig')
lecture_review_df.to_csv('./Downloads/lecture_review.csv',encoding='utf-8-sig')

In [39]:
load_lecture_info_df = pd.read_csv('./Downloads/lecture_info.csv',encoding='utf-8-sig',index_col=0)
load_lecture_info_df

,강의명,교수명,평균 평점,평점 참가자 수,평점 비율,과제량 비율,조모임 비율,학점 비율
0,경영정보시스템및실습,김종모,4.30,10,"[30, 70, 0, 0, 0]","[0, 70, 30]","[0, 40, 60]","[80, 20, 0]"
1,경영정보시스템및실습,손미애,3.24,96,"[16, 24, 38, 15, 8]","[3, 29, 68]","[17, 41, 43]","[9, 49, 42]"
2,경영과학(OR)및실습1,이규홍,3.55,11,"[45, 18, 9, 0, 27]","[0, 36, 64]","[82, 18, 0]","[9, 36, 55]"
3,경영과학(OR)및실습1,이희상,3.12,152,"[10, 26, 42, 9, 13]","[0, 13, 88]","[26, 73, 1]","[11, 69, 20]"
4,응용통계및연습1,이호우,3.90,156,"[42, 27, 19, 6, 7]","[0, 8, 92]","[90, 10, 0]","[22, 47, 31]"
5,응용통계및연습1,김윤배,1.33,3,NaN,"[0, 100, 0]","[100, 0, 0]","[0, 0, 100]"
6,응용통계및연습1,이종석,5.00,1,NaN,NaN,NaN,NaN
7,응용통계및연습2,강석호,4.24,41,"[46, 37, 12, 5, 0]","[7, 93, 0]","[83, 17, 0]","[37, 63, 0]"
8,응용통계및연습2,이종석,4.26,23,"[43, 39, 17, 0, 0]","[17, 83, 0]","[100, 0, 0]","[4, 96, 0]"
9,응용통계및연습2,이호우,4.35,34,"[76, 6, 3, 6, 9]","[0, 15, 85]","[88, 12, 0]","[68, 21, 12]"


In [40]:
load_lecture_review_df = pd.read_csv('./Downloads/lecture_review.csv',encoding='utf-8-sig',index_col=0)
load_lecture_review_df

,강의명,교수명,수강 시기,강평
0,경영정보시스템및실습,김종모,22년 1학기 수강자,이제는 손미애 교수님이 하시겟지만...\n김종모교수님 갓.... 그저 빛
1,경영정보시스템및실습,김종모,22년 1학기 수강자,교수님 말이 정말정말 빠르세요 필기가 불가능할 정도로 빠르심 근데 아캠에 강의저장을...
2,경영정보시스템및실습,김종모,22년 1학기 수강자,시험은 몇문제 빼고 괜찮은데 팀을 잘 만나야함. 교수님 굉장히 좋으시고 수업시간때 ...
3,경영정보시스템및실습,김종모,22년 1학기 수강자,교수님 말씀이 조금 빠르시지만 학점 미화.\n조별과제랑 시험 둘다 꽤 빡세서 엉엉 ...
4,경영정보시스템및실습,김종모,22년 1학기 수강자,성적은 잘 주시는 편인데 성적간 차이가 정말 촘촘하다. 중간기말 그저 그렇게 보고 ...
...,...,...,...,...
3016,창의적융합디자인,김용일,20년 1학기 수강자,그냥 흘러가는데로 있으면 됨. 시험도 없고 과제도 빠르게 하면 20분컷임. 하지만 ...
3017,창의적융합디자인,김용일,20년 1학기 수강자,학점비율은 학기 중이라 잘 모르겠는데 점수는 잘 주십니다 교수님 잘 가르치시고 젠틀...
3018,창의적융합디자인,김용일,20년 1학기 수강자,일단 본인은 인사캠 학생이라 1학기 때 창융디 직배걸려서 수원 안가도 되니 개꿀이라...
3019,창의적융합디자인,김용일,20년 1학기 수강자,그냥 재밌고 적당히 편하게 듣기에 좋은 수업. 하지만 내가 이 강의로부터 어떤 것들...
